In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
# criar spark session
spark = SparkSession \
    .builder \
    .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0,org.postgresql:postgresql:42.2.20') \
    .config('spark.hadoop.fs.s3a.access.key', '<your-access-key-id>') \
    .config('spark.hadoop.fs.s3a.secret.key', '<your-secret-key>') \
    .appName('spark etl - depreciacao') \
    .getOrCreate()

# ler arquivos no s3
df = spark.read.csv('s3a://custos-stn-bucket/depreciacao/depreciacao_0.csv', header=True, inferSchema=True)

In [3]:
# schema
df.printSchema()

root
 |-- co_natureza_juridica: integer (nullable = true)
 |-- ds_natureza_juridica: string (nullable = true)
 |-- co_organizacao_n1: integer (nullable = true)
 |-- ds_organizacao_n1: string (nullable = true)
 |-- co_organizacao_n2: integer (nullable = true)
 |-- ds_organizacao_n2: string (nullable = true)
 |-- co_organizacao_n3: integer (nullable = true)
 |-- ds_organizacao_n3: string (nullable = true)
 |-- an_lanc: integer (nullable = true)
 |-- me_lanc: integer (nullable = true)
 |-- id_conta_contabil: integer (nullable = true)
 |-- no_conta_contabil: string (nullable = true)
 |-- va_custo_depreciacao: double (nullable = true)



In [4]:
# contagem de linhas
df.count()

250

In [5]:
# emprimir primeiras 5 linhas
df.show(5)

+--------------------+--------------------+-----------------+--------------------+-----------------+--------------------+-----------------+-----------------+-------+-------+-----------------+--------------------+--------------------+
|co_natureza_juridica|ds_natureza_juridica|co_organizacao_n1|   ds_organizacao_n1|co_organizacao_n2|   ds_organizacao_n2|co_organizacao_n3|ds_organizacao_n3|an_lanc|me_lanc|id_conta_contabil|   no_conta_contabil|va_custo_depreciacao|
+--------------------+--------------------+-----------------+--------------------+-----------------+--------------------+-----------------+-----------------+-------+-------+-----------------+--------------------+--------------------+
|                   3|ADMINISTRACAO DIRETA|             2844|MINISTERIO DO TRA...|             3499|SUPERINTENDENCIA ...|               -9|    NAO SE APLICA|   2015|     11|        333110200|DEPRECIACAO DE BE...|                 0.0|
|                   4|           AUTARQUIA|              244|MIN

In [6]:
# criar view para spark sql
df.createOrReplaceTempView("depreciacao")

In [7]:

df_dp = spark.sql("""
                  select *
                  from depreciacao
                  """)

df_dp.printSchema()


root
 |-- co_natureza_juridica: integer (nullable = true)
 |-- ds_natureza_juridica: string (nullable = true)
 |-- co_organizacao_n1: integer (nullable = true)
 |-- ds_organizacao_n1: string (nullable = true)
 |-- co_organizacao_n2: integer (nullable = true)
 |-- ds_organizacao_n2: string (nullable = true)
 |-- co_organizacao_n3: integer (nullable = true)
 |-- ds_organizacao_n3: string (nullable = true)
 |-- an_lanc: integer (nullable = true)
 |-- me_lanc: integer (nullable = true)
 |-- id_conta_contabil: integer (nullable = true)
 |-- no_conta_contabil: string (nullable = true)
 |-- va_custo_depreciacao: double (nullable = true)



In [8]:
df_dp.show()

+--------------------+--------------------+-----------------+--------------------+-----------------+--------------------+-----------------+-----------------+-------+-------+-----------------+--------------------+--------------------+
|co_natureza_juridica|ds_natureza_juridica|co_organizacao_n1|   ds_organizacao_n1|co_organizacao_n2|   ds_organizacao_n2|co_organizacao_n3|ds_organizacao_n3|an_lanc|me_lanc|id_conta_contabil|   no_conta_contabil|va_custo_depreciacao|
+--------------------+--------------------+-----------------+--------------------+-----------------+--------------------+-----------------+-----------------+-------+-------+-----------------+--------------------+--------------------+
|                   3|ADMINISTRACAO DIRETA|             2844|MINISTERIO DO TRA...|             3499|SUPERINTENDENCIA ...|               -9|    NAO SE APLICA|   2015|     11|        333110200|DEPRECIACAO DE BE...|                 0.0|
|                   4|           AUTARQUIA|              244|MIN

In [9]:
df_dp.head(1) # Mostra o cabeçalho da primeira linha

[Row(co_natureza_juridica=3, ds_natureza_juridica='ADMINISTRACAO DIRETA', co_organizacao_n1=2844, ds_organizacao_n1='MINISTERIO DO TRABALHO E PREVIDENCIA SOCIAL', co_organizacao_n2=3499, ds_organizacao_n2='SUPERINTENDENCIA REGIONAL DO TRABALHO E EMPREGO EM RONDONIA', co_organizacao_n3=-9, ds_organizacao_n3='NAO SE APLICA', an_lanc=2015, me_lanc=11, id_conta_contabil=333110200, no_conta_contabil='DEPRECIACAO DE BENS IMOVEIS', va_custo_depreciacao=0.0)]

In [10]:
df_dp.describe(["co_natureza_juridica","ds_natureza_juridica","co_organizacao_n1","ds_organizacao_n1","co_organizacao_n2"]).show()

+-------+--------------------+--------------------+------------------+--------------------+------------------+
|summary|co_natureza_juridica|ds_natureza_juridica| co_organizacao_n1|   ds_organizacao_n1| co_organizacao_n2|
+-------+--------------------+--------------------+------------------+--------------------+------------------+
|  count|                 250|                 250|               250|                 250|               250|
|   mean|               3.072|                null|         12867.292|                null|          21107.56|
| stddev|  0.9625601738870216|                null|26381.644497496833|                null|36599.926589980576|
|    min|                   1|ADMINISTRACAO DIRETA|                -9|CASA CIVIL DA PRE...|                -9|
|    max|                   6|SOCIEDADE DE ECON...|            208423|VICE-PRESIDENCIA ...|            122391|
+-------+--------------------+--------------------+------------------+--------------------+------------------+



In [11]:
df_dp.describe(["ds_organizacao_n2","co_organizacao_n3","ds_organizacao_n3","an_lanc","me_lanc","id_conta_contabil"]).show()

+-------+--------------------+-----------------+-----------------+-------+-----------------+-----------------+
|summary|   ds_organizacao_n2|co_organizacao_n3|ds_organizacao_n3|an_lanc|          me_lanc|id_conta_contabil|
+-------+--------------------+-----------------+-----------------+-------+-----------------+-----------------+
|  count|                 250|              250|              250|    250|              250|              250|
|   mean|                null|             -9.0|             null| 2015.0|            7.292|    3.331205536E8|
| stddev|                null|              0.0|             null|    0.0|3.315827929708921| 31878.4023625472|
|    min|AGENCIA BRASILEIR...|               -9|    NAO SE APLICA|   2015|                1|        333110100|
|    max|VALEC - ENGENHARI...|               -9|    NAO SE APLICA|   2015|               12|        333310100|
+-------+--------------------+-----------------+-----------------+-------+-----------------+-----------------+



In [12]:
df_dp.describe(["no_conta_contabil","va_custo_depreciacao"]).show()

+-------+--------------------+--------------------+
|summary|   no_conta_contabil|va_custo_depreciacao|
+-------+--------------------+--------------------+
|  count|                 250|                 250|
|   mean|                null|  189367.94784000007|
| stddev|                null|   538971.3656610626|
|    min|AMORTIZACAO DE IM...|         -3162462.47|
|    max|            EXAUSTAO|          2772203.09|
+-------+--------------------+--------------------+



In [13]:
df_dp.select('ds_natureza_juridica').distinct().rdd.map(lambda r: r[0]).collect()
# Util para conhecer classes discretas de uma coluna categórica.
# Neste caso mostra as organizacoes presentes nestes dados

['AUTARQUIA',
 'SOCIEDADE DE ECONOMIA MISTA',
 'FUNDACAO PUBLICA',
 'ADMINISTRACAO DIRETA',
 'EMPRESA PUBLICA']

In [14]:
df_dp.select('no_conta_contabil').distinct().rdd.map(lambda r: r[0]).collect()
# Util para conhecer classes discretas de uma coluna categórica.
# Neste caso mostra as modalidades de aplicacao presentes nestes dados

['DEPRECIACAO DE BENS IMOVEIS',
 'AMORTIZACAO DE INTANGIVEL',
 'AMORTIZACAO DE IMOBILIZADO',
 'DEPRECIACAO DE BENS MOVEIS',
 'EXAUSTAO']

In [15]:
df_dp = spark.sql("""
                  select distinct *
                  from depreciacao
                  """)
df_dp.count() # Verificar se tem linhas duplicadas

# Retorna 250, isso quer dizer que as linhas sao todas diferentes

250

In [16]:
df_dp.head(1)

[Row(co_natureza_juridica=4, ds_natureza_juridica='AUTARQUIA', co_organizacao_n1=1930, ds_organizacao_n1='MINISTERIO DA PREVIDENCIA SOCIAL', co_organizacao_n2=1934, ds_organizacao_n2='INSTITUTO NACIONAL DO SEGURO SOCIAL', co_organizacao_n3=-9, ds_organizacao_n3='NAO SE APLICA', an_lanc=2015, me_lanc=12, id_conta_contabil=333110100, no_conta_contabil='DEPRECIACAO DE BENS MOVEIS', va_custo_depreciacao=3371.18)]

In [17]:
df_dp_unique = spark.sql("""
                  select co_natureza_juridica || '-' || ds_natureza_juridica || '-' || co_organizacao_n1 || '-' || 
                  ds_organizacao_n1 || '-' ||co_organizacao_n2 || '-' || ds_organizacao_n2 ||'-' || 
                  co_organizacao_n3 || '-' || ds_organizacao_n3 || '-' || me_lanc || '-'|| an_lanc || va_custo_depreciacao ||
                  no_conta_contabil, 
                  count(1)
                  from depreciacao
                  group by 1
                  having count(1) > 1
                  """)
# Encontrando a chave primária através de:
# 1- co_natureza_juridica
# 2- ds_natureza_juridica
# 3- co_organizacao_n1
# 4- ds_organizacao_n1
# 5- co_organizacao_n2
# 6- ds_organizacao_n2
# 7- co_organizacao_n3
# 8- ds_organizacao_n3
# 9- me_lanc
# 10- an_lanc
# 11- va_custo_depreciacao
# 12- no_conta_contabil

df_dp_unique.count()

0

In [18]:
df_dp.groupBy(["co_natureza_juridica","ds_natureza_juridica"]).count().sort("count", ascending=True).show()
# Mostra a quantidade de descricao por natureza juridica

+--------------------+--------------------+-----+
|co_natureza_juridica|ds_natureza_juridica|count|
+--------------------+--------------------+-----+
|                   6|SOCIEDADE DE ECON...|    2|
|                   2|    FUNDACAO PUBLICA|   20|
|                   1|     EMPRESA PUBLICA|   27|
|                   4|           AUTARQUIA|   86|
|                   3|ADMINISTRACAO DIRETA|  115|
+--------------------+--------------------+-----+



In [21]:
df.write \
  .format("jdbc") \
  .option("url", "jdbc:postgresql://<database-ip>:<port>/custos_stn") \
  .option("dbtable", "custos_stn.depreciacao") \
  .option("user", "postgres") \
  .option("driver", "org.postgresql.Driver") \
  .option("password", "postgres")\
  .save()